# MISP

In [ ]:
%pip install pymisp > /dev/null

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
from dotenv import dotenv_values

In [ ]:
from surianalytics.connectors import getGitRoot, check_str_bool

In [ ]:
import os
import shutil

In [ ]:
config = dotenv_values(os.path.join(getGitRoot(), ".env") if shutil.which("git") is not None else os.path.join(os.path.expanduser("~"), ".env"))

In [ ]:
from pymisp import PyMISP

In [ ]:
misp = PyMISP(config["MISP_HOST"],
              config["MISP_TOKEN"],
              check_str_bool(config["MISP_TLS_VERIFY"]),
              debug=False)

In [ ]:
result = misp.search(controller='attributes', timestamp="1d", type_attribute="domain", category="Network activity", pythonify=False)

In [ ]:
import pandas as pd

In [ ]:
DF_ATTR = pd.json_normalize(result["Attribute"])
len(DF_ATTR)

In [ ]:
DF_ATTR.groupby("type").agg({"value": "nunique",
                             "event_id": "nunique"})

In [ ]:
DF_ATTR.groupby(["Event.info", "event_id", "type"]).agg({"value": ["unique", "nunique"]})

# Scan for IoC values via scirius

In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
c = RESTSciriusConnector()

In [ ]:
DAYS = 7

In [ ]:
c.set_query_delta(hours=24 * DAYS)

In [ ]:
c.set_page_size(10000)

In [ ]:
DF_EVENTS = c.retrosearch(domains=list(DF_ATTR.value.unique()))
len(DF_EVENTS)

# Report domain matches

## Group by value

In [ ]:
(
    DF_EVENTS
    .groupby(["ioc.value.match"])
    .agg({
        "timestamp": ["min", "max"],
        "src_ip": ["unique", "nunique"],
        "dest_ip": ["unique", "nunique"],
        "tls.ja3.hash": ["unique"],
        "tls.ja3s.hash": ["unique"],
        "http.status": ["unique"],
        "http.url": ["unique"]
    })
)

## Group by time

In [ ]:
BIN_SIZE = "1h"

In [ ]:
(
    DF_EVENTS
    .groupby([pd.Grouper(key="timestamp", freq=BIN_SIZE)])
    .agg({
        "timestamp": ["min", "max"],
        "ioc.value.match": ["unique", "nunique", "count"],
        "src_ip": ["unique", "nunique"],
        "dest_ip": ["unique", "nunique"],
        "tls.ja3.hash": ["unique"],
        "tls.ja3s.hash": ["unique"],
        "http.status": ["unique"],
        "http.url": ["unique"],
    })
)

# Plot results

## By event type

In [ ]:
import hvplot.pandas

In [ ]:
DF_AGG_TIME_PLOT = (
    DF_EVENTS
    .groupby([pd.Grouper(key="timestamp", freq=BIN_SIZE), "event_type"])
    .agg({
        "ioc.value.match": ["count"],
    })
    .reset_index()
)

In [ ]:
DF_AGG_TIME_PLOT.columns = ["timestamp", "event_type", "count"]

In [ ]:
(
    DF_AGG_TIME_PLOT
    .pivot(index="timestamp", columns=["event_type"], values="count")
    .hvplot
    .line(width=900, height=400)
)

## By Domain

In [ ]:
DF_AGG_TIME_PLOT_CPX = (
    DF_EVENTS
    .groupby([pd.Grouper(key="timestamp", freq=BIN_SIZE), "ioc.value.match"])
    .agg({
        "ioc.value.match": ["count"]
    })
    .reset_index()
)

In [ ]:
DF_AGG_TIME_PLOT_CPX

In [ ]:
DF_AGG_TIME_PLOT_CPX.columns = ["timestamp", "domain", "count"]

In [ ]:
(
    DF_AGG_TIME_PLOT_CPX
    .pivot(index="timestamp", columns="domain", values="count")
    .hvplot
    .scatter(width=900, height=400)
)